In [1]:
import syft as sy


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
node = sy.orchestra.launch(name="test-jobs", dev_mode=True, reset=True, create_producer=True, queue_port=5556, n_consumers=3)
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <test-jobs: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
x = sy.ActionObject.from_obj([1,2,3])
x_ptr = x.send(client)

In [4]:
@sy.syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    import random
    sleep(random.random()*5)
    print("done")
    return batch + 1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [6]:
@sy.syft_function()
def aggregate_job(job_results):
    print([(res, type(res))for res in job_results])
    results = [res for res in job_results]
    return sum(results)

SyftSuccess: Syft function 'aggregate_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.submit(aggregate_job)

SyftSuccess: User Code Submitted

In [8]:
@sy.syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    job_results = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        job_results.append(batch_job.result)
    print("starting aggregation")
    print("Done")
    reduce_job = domain.launch_job(aggregate_job, job_results=job_results)
    return reduce_job.result

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [9]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-jobs


SyftSuccess: Request ee21e794e1e64d56b5f881108a47436a changes applied

In [10]:
from time import sleep
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(10)

24/11/23 12:34:14 FUNCTION LOG (2574b19651f14d0993194fcee98761e5): Launching jobs
24/11/23 12:34:15 FUNCTION LOG (2574b19651f14d0993194fcee98761e5): starting aggregation
24/11/23 12:34:15 FUNCTION LOG (2574b19651f14d0993194fcee98761e5): Done
24/11/23 12:34:16 FUNCTION LOG (46b45ee456b944dea6f7cb79d270ee41): starting batch 1
24/11/23 12:34:16 FUNCTION LOG (b0d88895c5a74a9aaed6a1b77bc88985): starting batch 2
24/11/23 12:34:17 FUNCTION LOG (8c73ffcb58644a538a085cbc7800c92e): starting batch 3
24/11/23 12:34:18 FUNCTION LOG (46b45ee456b944dea6f7cb79d270ee41): done
24/11/23 12:34:18 FUNCTION LOG (b0d88895c5a74a9aaed6a1b77bc88985): done
24/11/23 12:34:22 FUNCTION LOG (8c73ffcb58644a538a085cbc7800c92e): done
24/11/23 12:34:23 FUNCTION LOG (4a3edb89a7ec41628cea08f62d0fff99): [(2, <class 'int'>), (3, <class 'int'>), (4, <class 'int'>)]


In [14]:
result = job.resolve.get().get()

In [15]:
result == 9

True